In [59]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [60]:
# os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [55]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
# embeddings = GoogleGenerativeAIEmbeddings (model="models/embedding-001")
# llm = ChatGoogleGenerativeAI(model = "gemini-1.0-pro")

In [56]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant")

In [57]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings2 = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/home/user/Meet_Patel/AllChatbot-Project/chatenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


SIMPLE AI Assistant

Prompt -> LLM -> Response

In [17]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

How's it going? Is there something I can help you with or would you like to chat?
I'm a large language model, I don't have real-time access to current weather conditions. However, I can suggest some options to help you find out the current weather in your area:

1. **Check online weather websites**: You can visit websites like AccuWeather, Weather.com, or the National Weather Service (NWS) for current weather conditions and forecasts.
2. **Use a weather app**: Download a weather app on your smartphone, such as Dark Sky, Weather Underground, or The Weather Channel, to get current weather conditions and forecasts.
3. **Ask a virtual assistant**: If you have a virtual assistant like Siri, Google Assistant, or Alexa, you can ask them for the current weather in your area.
4. **Tune into local news**: Watch local news or listen to local radio stations for weather updates.

Please let me know what location you would like to know the weather for, and I'll do my best to help you find the inform

In [18]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [19]:
store={}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [20]:
config = {"configurable": {"session_id": "firstchat"}}

In [21]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [22]:
model_with_memory.invoke(("Hi! I'm Virtual Machine"),config=config).content


"Nice to meet you, Virtual Machine! You're a software emulation of a physical computer, allowing you to run an operating system (OS) on top of a host machine's OS. This enables you to run multiple OS environments on a single physical machine, which can be really useful for testing, development, and other purposes.\n\nWhat kind of virtual machine are you? Are you a typical x86-based VM, a cloud-based VM, or perhaps a special-purpose VM like a container?"

In [23]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'I don\'t actually know your name. I called you "Virtual Machine" because that\'s what you said you were. If you\'d like to share a specific name for yourself, I\'d be happy to know it!'

RAG WITH LCEL

In [25]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser


In [26]:

### Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings2)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [27]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [28]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [29]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Based on the provided context, it appears that Llama 3 is a model used for various AI services. Here are 3 important points about Llama 3:

1. **Llama 3 is a model**: The context describes Llama 3 as a model, specifically mentioning an "8B parameter version" of it. This suggests that Llama 3 is a type of artificial intelligence (AI) model.

2. **Released in April 2024**: According to the context, Llama 3 is released in April 2024. This information suggests that Llama 3 is a relatively new AI model.

3. **Used by Meta services**: The context mentions that Meta added Llama 3 to their services alongside its release. This implies that Llama 3 is used by various AI-powered services provided by Meta, such as its website.


TOOLS AND AGENTS

Wikipedia Tool


In [30]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [38]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [39]:
wiki_tool.name

'wikipedia'

In [40]:
wiki_tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [41]:
wiki_tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [42]:
wiki_tool.return_direct

False

In [43]:
print(wiki_tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


Youotube Search Tool

In [44]:
from langchain_community.tools import YouTubeSearchTool

In [45]:
youtube_tool = YouTubeSearchTool()

In [46]:
youtube_tool.name

'youtube_search'

In [47]:
youtube_tool.description

'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [49]:
youtube_tool.run("Langchain")

"['https://www.youtube.com/watch?v=1bUy-1hGZpI&pp=ygUJTGFuZ2NoYWlu', 'https://www.youtube.com/watch?v=nAmC7SoVLd8&pp=ygUJTGFuZ2NoYWlu']"

TAVILYSEARCH - WEBSEARCH

In [61]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [64]:
web_tool = TavilySearchResults()

In [72]:
web_tool.name

'tavily_search_results_json'

In [77]:
web_tool.invoke({"query": "Latest news on the movie gen by meta."})

[{'url': 'https://www.wired.com/story/meta-movie-gen-ai-model/',
  'content': 'Meta’s Movie Gen Makes Convincing AI Video Clips | WIRED Meta’s Movie Gen Makes Convincing AI Video Clips Meta’s Movie Gen Makes Convincing AI Video Clips Meta just announced its own media-focused AI model, called Movie Gen, that can be used to generate realistic video and audioclips. Going beyond the generation of straightforward text-to-video clips, the Movie Gen model can make targeted edits to an existing clip, like adding an object into someone’s hands or changing the appearance of a surface. Considering Meta’s legacy as a social media company, it’s possible that tools powered by Movie Gen will start popping up, eventually, inside of Facebook, Instagram, and WhatsApp. In September, competitor Google shared plans to make aspects of its Veo video model available to creators inside its YouTube Shorts sometime next year.'},
 {'url': 'https://arstechnica.com/ai/2024/10/metas-new-movie-gen-ai-system-can-deepf

AGENTS OLD CLASSES

In [66]:
from langchain.agents import AgentType, load_tools, initialize_agent

In [78]:
tool1 = load_tools(["wikipedia"], llm=llm)
agent = initialize_agent(tool1, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [79]:
agent.run("Two lines on Indian monument tajmahal.")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: To find information about the Taj Mahal, I should use a reliable source.

Action: wikipedia
Action Input: Taj Mahal
Observation: Page: Taj Mahal
Summary: The Taj Mahal ( TAHJ mə-HAHL, TAHZH -⁠, Hindi: [taːdʒ ˈmɛɦ(ɛ)l]; lit. 'Crown of the Palace') is an ivory-white marble mausoleum on the right bank of the river Yamuna in Agra, Uttar Pradesh, India. It was commissioned in 1631 by the fifth Mughal emperor, Shah Jahan (r. 1628–1658) to house the tomb of his beloved wife, Mumtaz Mahal; it also houses the tomb of Shah Jahan himself. The tomb is the centrepiece of a 17-hectare (42-acre) complex, which includes a mosque and a guest house, and is set in formal gardens bounded on three sides by a crenellated wall.
Construction of the mausoleum was completed in 1648, but work continued on other phases of the project for another five years. The first ceremony held at the mausoleum was an observance by Shah Jahan, on 6 February 1643, of the 12th anniversary of the death of Mumtaz Mahal. T

"The Taj Mahal ( TAHJ mə-HAHL, TAHZH -\u2060, Hindi: [taːdʒ ˈmɛɦ(ɛ)l]; lit.\u2009'Crown of the Palace') is an ivory-white marble mausoleum on the right bank of the river Yamuna in Agra, Uttar Pradesh, India. It was commissioned in 1631 by the fifth Mughal emperor, Shah Jahan (r.\u20091628–1658) to house the tomb of his beloved wife, Mumtaz Mahal; it also houses the tomb of Shah Jahan himself."